### Наивный Байесовский классификатор

"Наивный" &ndash; т.к. предполагает, что комбинируемые вероятности независимы.

<!-- P(A|B) = P(B|A * P(A) / P(B) -->
$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

Допустим, у нас есть набор документов, принадлежащих разным категориям.

Попробуем определить вероятность того, что какой-то документ принадлежит категории, т.е. вероятность $P(категория|документ)$

<!-- P(кат|док) = P(док|кат) * P(кат) / P(док) -->
$$P(cat|doc) = \frac{P(doc|cat) \cdot P(cat)}{P(doc)}$$

Оценим вероятность появления документа в данной категории $P(док|кат)$ как произведение вероятностей появления каждого слова в категории:

<!-- P(док|кат) = П(P(слово|кат)) -->
$$P(doc|cat) = \prod_{word}{P(word|cat)}$$

<!-- P(слово|кат) = C(слово, кат) / Σ(С(слово(i), кат)) -->
$$P(word|cat) = \frac{C(word, cat)}{L(cat)}$$

где $L$ &ndash; длина категории в словах.

Оценим $P(кат)$ как вероятность попадания случайно выбранного доумента в конкретную категорию.

<!-- P(кат) = C(док, кат) / C(док) -->
$$P(cat) = \frac{C(doc, cat)}{C(doc)}$$

$P(док)$ не зависит от категорий и для всех сравниваемых величин одинакова, поэтому можно её не учитывать. Тогда искомая категория:

$$hypothesis = \underset{cat}{\text{arg max}}{\prod_{word}{\left( \frac{ C(word, cat)}{L(cat)} \right)} \cdot \frac{C(doc, cat)}{C(doc)}}$$

1. Будет удобнее, если вы будете хранить вероятности не в прямом представлении, а в логарифмическом (основание логарифма может быть любым). Это позволяет:

<details>
    <summary></summary>

    а) избежать underflow
    б) заменить умножение на сложение
</details>

2. Понятие стоп-слов: частотные слова, которые встречаются во всех категориях. Такие слова нужно убрать из текстов.

In [ ]:
import nltk
nltk.download('stopwords')

In [3]:
from nltk.corpus import stopwords
" ".join(stopwords.words("russian"))

'и в во не что он на я с со как а то все она так его но да ты к у же вы за бы по только ее мне было вот от меня еще нет о из ему теперь когда даже ну вдруг ли если уже или ни быть был него до вас нибудь опять уж вам ведь там потом себя ничего ей может они тут где есть надо ней для мы тебя их чем была сам чтоб без будто чего раз тоже себе под будет ж тогда кто этот того потому этого какой совсем ним здесь этом один почти мой тем чтобы нее сейчас были куда зачем всех никогда можно при наконец два об другой хоть после над больше тот через эти нас про всего них какая много разве три эту моя впрочем хорошо свою этой перед иногда лучше чуть том нельзя такой им более всегда конечно всю между'

3. Иногда при делении на джва класса нам нужно иметь возможность сказать "не знаю". Тогда вычислим отношение между вероятностями принадлежности к двум категориям. Если оно меньше какого-то заданного порога, то однозначно принять решение нельзя.

4. Если в определяемом тексте нам встречается слово, которого не было ни в одной из категорий, мы можем его просто пропустить (оно не будет влиять на выбор категории). Если оно было только в одной категории, то мы можем в другой категории сделать вероятность этого слова или нулём (тогда все тексты с этим словом автоматически будут попадать в другую категорию), или специально заданным очень маленьким числом.

5. Скорее всего, для такой классификации конкретные словоформы нам не очень важны. Поэтому воспользуемся pymorphy2 для определения исходной формы каждого слова.

6. Модуль pickle для сериализации (хранения) данных

In [ ]:
!wget https://pkholyavin.github.io/year4programming/conference_stud_clean.pkl

In [ ]:
import pickle

with open("conference_stud_clean.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
with open("new_pickle.pkl", "wb") as f:
    pickle.dump(data, f)

Посмотрим на список секций:

In [ ]:
set([i["section"] for i in data])

{'Антрополингвистика: Человек, Язык, Культура',
 'Балканистика. Византинистика. Неоэллинистика',
 'Балтистика',
 'Библеистика',
 'Будетляне. Гипотеза в филологии: TED TALKS',
 'Будетляне. Гипотеза в филологии: научные бои',
 'Будетляне. Гипотеза в филологии: научный слэм',
 'Грамматика (романо-германская филология)',
 'Грамматика и семантика русского языка',
 'Дискурс и текст',
 'История зарубежных литератур',
 'История и диалектология русского языка',
 'История и теория русского и западноевропейского стиха',
 'История русского языка',
 'Кино|Текст',
 'Киноперевод',
 'Классическая филология',
 'Коллоквиалистика (анализ устной речи)',
 'Компьютерная и прикладная лингвистика',
 'Лексикология (романо-германская филология)',
 'Лексикология и стилистика русского языка',
 'Лингвометодические основы описания и изучения русского языка как иностранного',
 'Литература в междисциплинарной перспективе',
 'Междисциплинарная постерная сессия',
 'Народная культура в древнем и новом слове',
 'Общее яз

Задание: обучить классификатор на корпусе текстов студенческой конференции (две разные секции или группы секций &ndash; например, литературоведение и лингвистика). Взять текст, не входящий в обучающую выборку, и попробовать определить его вероятность по каждой из двух моделей. Где вероятность больше, той секции, скорее всего, он и будет принадлежать.

**1. Создать обучающие выборки**

1.1. Отобрать тексты для каждой секции

1.2. Каждый текст разбить на слова, произвести лемматизацию

**2. Обучить классификатор**

2.1. Для каждой категории вычислить $P(cat)$

2.2. Составить список слов (лексикон)

2.3. Для каждого слова и каждой категории вычислить $P(word|cat)$

**3. Протестировать классификатор на тексте, не входящем в обучающую выборку**

**4. Попробуйте построить систему не с двумя, а с тремя классами (или больше).**